In [ ]:
import numpy as np
np.random.seed(1)

from keras.models import Sequential
from keras.layers import Dense, Embedding, Flatten, Input
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.optimizers import Adam
from sklearn.preprocessing import OneHotEncoder


In [ ]:
# define documents
docs = ['king is man',
'a king marrys queen',
'a queen marrys king',
'some unrelated workds'
'queen is woman']
docs

In [ ]:
vocab_size = 50

In [ ]:
oh = np.array(one_hot('king queen man woman unrelated', vocab_size)).reshape(-1,1)
oh

In [ ]:
enc = OneHotEncoder()
enc.fit(np.array(range(50)).reshape(-1,1))
oh_enc = enc.transform(oh).toarray()
oh_enc

In [ ]:
encoded_docs = [one_hot(d, vocab_size) for d in docs]
encoded_docs 

In [ ]:
max_length = 24
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
padded_docs 

In [ ]:
tuples = np.empty((0, 2))
for padded_doc in padded_docs:
    length = len(padded_doc)
    for i in range(length):
        if padded_doc[i] != 0:
            if i<length-1 & padded_doc[i+1] != 0:
                tuples = np.append(tuples, [[padded_doc[i],padded_doc[i+1]]], axis=0) 
                if i<length-2 & padded_doc[i+2] != 0:
                    tuples = np.append(tuples, [[padded_doc[i],padded_doc[i+2]]], axis=0) 
            if i > 0:
                tuples = np.append(tuples, [[padded_doc[i],padded_doc[i-1]]], axis=0) 
                if i > 1:
                    tuples = np.append(tuples, [[padded_doc[i],padded_doc[i-2]]], axis=0) 

    
print (tuples.shape)
tuples



In [ ]:
onehotlabels_x = enc.transform(tuples[:,0].reshape(-1, 1)).toarray()

print  (onehotlabels_x.shape)
print ('---')
print (onehotlabels_x)
print ('---')
print (onehotlabels_x[0])

In [ ]:
onehotlabels_y = enc.transform(tuples[:,1].reshape(-1, 1)).toarray()

print  (onehotlabels_y.shape)
print ('---')
print (onehotlabels_y)
print ('---')
print (onehotlabels_y[0])

In [ ]:
model = Sequential()

input = Dense(50, input_shape=(50,), activation='relu')
model.add(input)
bottleneck = Dense(2, activation='relu')
model.add(bottleneck)
model.add(Dense(50, activation='softmax'))
# compile the model
model.compile(optimizer="adam", loss='binary_crossentropy', metrics=['acc'])
# summarize the model
print(model.summary())

# fit the model
model.fit(onehotlabels_x, onehotlabels_y, epochs=500, verbose=1)

In [ ]:
model = Sequential()
model.add(input)
model.add(bottleneck)
model.compile(optimizer=Adam(lr=0.001), loss='binary_crossentropy', metrics=['acc'])
test = model.predict(oh_enc)
print (test)


In [ ]:
model = Sequential()
model.add(Embedding(1000, 3, input_length=5))
model.compile('rmsprop', 'mse')

input_array = np.random.randint(1000, size=(3, 5)) # 3 documents, 5 words per document
input_array


In [ ]:
output_array = model.predict(input_array)
print (output_array.shape)
print (output_array)

In [ ]:
# define documents
docs = ['Well done!',
'Good work',
'Great effort',
'nice work',
'Excellent!',
'Really Weak',
'Poor effort!',
'not good',
'poor work',
'Could have done better.']
# define class labels
labels = [1,1,1,1,1,0,0,0,0,0]

In [ ]:
# integer encode the documents
vocab_size = 50
encoded_docs = [one_hot(d, vocab_size) for d in docs]
print(encoded_docs)

In [ ]:
# pad documents to a max length of 4 words
max_length = 4
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
print(padded_docs)

In [ ]:
model = Sequential()
model.add(Embedding(vocab_size, 8, input_length=max_length))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
# summarize the model
print(model.summary())

In [ ]:
# fit the model
model.fit(padded_docs, labels, epochs=50, verbose=0)
# evaluate the model
loss, accuracy = model.evaluate(padded_docs, labels, verbose=0)
print('Accuracy: %f' % (accuracy*100))

In [ ]:
import numpy as np
np.hstack((model.predict(padded_docs),np.array(labels).reshape(10,1)))